In [1]:
import os 
import numpy as np
import pandas as pd
import tensorflow as tf
os.getcwd()
from google.colab import drive
drive.mount('/content/gdrive')

from iteration_utilities import duplicates
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
train_x = pd.read_csv("/content/train_data_rna-seq.csv")
test_x = pd.read_csv("/content/test_data_rna-seq_v1.csv")
train_patient = pd.read_csv("/content/train_data_clinical_patient.csv") #train_data_clinical_patient.csv - the training set (임상정보) - 환자와 관련된 임상 데이터

In [3]:
## clinical_patient에서 환자이름명 추출
train_patient = train_patient.rename(index = train_patient["PATIENT_ID"])
patientID = list(train_patient["PATIENT_ID"].values)

In [4]:
test_x = test_x.dropna(axis=0)
test_x = test_x.transpose()
test_x = test_x.rename(lambda label:label[8:])
test_x['sort'] = test_x.index
test_x = test_x.iloc[2:] ##2번째 행부터
test_x = test_x.astype(np.float64) ## 부동소수형 
test_x = test_x.sort_values(by = 'sort', ascending=True)
test_x = test_x.drop(columns='sort')

In [5]:
train_x = train_x.dropna(axis=0)
train_x = train_x.transpose()
train_x = train_x[2:]
train_x = train_x.rename(lambda label:label[:-3]) # train_data_rna-seq에서 [:-3]을 수행하면 clinical_patient 환자명과 동일해짐 
train_x_id = list(train_x.index)

In [6]:
train_x.shape

(2763, 16328)

In [7]:
test_x.shape

(74, 16328)

In [8]:
drop = []
for i in train_x_id:
    if i not in patientID:
        drop.append(i)
train_x = train_x.drop(index=drop) ## test_x (x,)값 맞추기

In [9]:
train_patient.shape

(2697, 37)

In [10]:
train_x.shape

(2607, 16328)

In [11]:
chk_index_list = list(duplicates(list(train_patient.index))) ## 중복값제거
train_patient = train_patient.drop(index= chk_index_list)
chk_index_list = list(duplicates(list(train_x.index)))
train_x = train_x.drop(index=chk_index_list)

In [12]:
#train_x.shape #2603개

In [13]:
#train_patient.shape #2697개

In [14]:
patientID = list(train_x.index) # 2697개라 2603으로 맞추기
train_x_id = list(train_patient.index)
for i in train_x_id:
    if i not in patientID:
        train_patient = train_patient.drop(index=i) ## 값이 같아질떄까지 드롭

In [15]:
train_patient.shape

(2603, 37)

In [16]:
train_x.shape

(2603, 16328)

In [17]:
cancer = train_patient['CANCER_TYPE_ACRONYM']
x = train_x.iloc[:,:500] ##(,y)값 맞추기

In [18]:
#x

In [19]:
test_x = test_x.iloc[:,:500]
y = cancer

In [20]:
test_x.shape

(74, 500)

In [21]:
y.shape

(2603,)

In [22]:
x.shape

(2603, 500)

In [23]:
encode = LabelEncoder()

## 인코딩 
encode.fit(y)
y_encoded = encode.transform(y)

for label in np.unique(y):
    if label not in encode.classes_: 
        encode.classes_ = np.append(encode.classes_, label)
y_encoded = encode.transform(y)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size= 0.1) #0.1로 자름

In [25]:
train = classifier = RandomForestClassifier(n_estimators=500, max_depth=50,random_state=10)
train.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [26]:
score = train.score(x_test, y_test)
print(score)

0.9885057471264368


In [27]:
predict = train.predict(test_x)

predict = encode.inverse_transform(predict) #숫자 -> 암 형식으로 인코딩 

In [28]:
predict = pd.DataFrame(predict)
final_data = pd.read_csv('/content/sample_submission.csv')
final_data = final_data.drop(['expected'],axis=1)
final_data['expected']= predict
final_data.to_csv("/content/Result_file.csv",index = False)